![Class Relationship](figs/classrelationship.png)

In [ ]:
from fuzzingbook.Fuzzer import Fuzzer
from typing import List, Set, Any, Tuple, Dict, Union
from fuzzingbook.GreyboxFuzzer import Seed, Mutator, PowerSchedule
import random


# 我们的 AdvancedMutationFuzzer 类是 MutationFuzzer 类的高级和参数化版本，该类来自基于突变的模糊测试章节。它也继承自 Fuzzer 类。目前，我们只需要知道函数 fuzz()，它返回生成的输入，以及函数 runs()，它执行 fuzz() 指定次数。对于我们的 AdvancedMutationFuzzer 类，我们重写了函数 fuzz()。

class AdvancedMutationFuzzer(Fuzzer):
    """Base class for mutation-based fuzzing."""

    def __init__(self, seeds: List[str],
                 mutator: Mutator,
                 schedule: PowerSchedule) -> None:
        """Constructor.
        `seeds` - a list of (input) strings to mutate.
        `mutator` - the mutator to apply.
        `schedule` - the power schedule to apply.
        """
        self.seeds = seeds
        self.mutator = mutator
        self.schedule = schedule
        self.inputs: List[str] = []
        self.reset()

    def reset(self) -> None:
        """Reset the initial population and seed index"""
        self.population = list(map(lambda x: Seed(x), self.seeds))
        self.seed_index = 0

    def create_candidate(self) -> str:
        """Returns an input generated by fuzzing a seed in the population"""
        seed = self.schedule.choose(self.population)

        # Stacking: Apply multiple mutations to generate the candidate
        candidate = seed.data
        trials = min(len(candidate), 1 << random.randint(1, 5))
        for i in range(trials):
            candidate = self.mutator.mutate(candidate)
        return candidate

    def fuzz(self) -> str:
        """Returns first each seed once and then generates new inputs"""
        if self.seed_index < len(self.seeds):
            # Still seeding
            self.inp = self.seeds[self.seed_index]
            self.seed_index += 1
        else:
            # Mutating
            self.inp = self.create_candidate()

        self.inputs.append(self.inp)
        return self.inp
    


seed_input = "ELonMusk"
mutation_fuzzer = AdvancedMutationFuzzer([seed_input], Mutator(), PowerSchedule())
print(mutation_fuzzer.fuzz())
print(mutation_fuzzer.fuzz())
print(mutation_fuzzer.fuzz())